In [1]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 9.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
thinc 8.3.6 requ

In [2]:
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 43.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
import requests
from bs4 import BeautifulSoup

import nltk
from nltk.corpus import stopwords

from gensim.models import Word2Vec

from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import seaborn as sns

import spacy
nlp = spacy.load('fr_core_news_sm')

# Téléchargements nécessaires
nltk.download('stopwords')

# 1. Récupérer des descriptions de plats depuis une page web (exemple)
url = 'https://huggingface.co/datasets/worldcuisines/vqa#dish-selection'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Supposons que les descriptions sont dans des balises <p>
descriptions = [p.get_text() for p in soup.find_all('p')]

# 2. Prétraitement du texte
nlp = spacy.load('fr_core_news_sm')  # Modèle français, ou 'en_core_web_sm' pour anglais

def preprocess_text(text):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return tokens

processed_descriptions = [preprocess_text(desc) for desc in descriptions]

# 3. Extraction de caractéristiques avec Word2Vec (Gensim)
model_w2v = Word2Vec(sentences=processed_descriptions, vector_size=100, window=5, min_count=2, workers=4)

# Obtenir une représentation vectorielle moyenne pour chaque description
import numpy as np

def get_sentence_vector(tokens, model):
    valid_tokens = [token for token in tokens if token in model.wv]
    if not valid_tokens:
        return np.zeros(model.vector_size)
    return np.mean([model.wv[token] for token in valid_tokens], axis=0)

vectors = np.array([get_sentence_vector(tokens, model_w2v) for tokens in processed_descriptions])

# 4. Clustering avec scikit-learn
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(vectors)

# 5. Visualisation avec PCA
pca = PCA(n_components=2)
reduced_vectors = pca.fit_transform(vectors)

plt.figure(figsize=(8,6))
sns.scatterplot(x=reduced_vectors[:,0], y=reduced_vectors[:,1], hue=clusters, palette='Set2')
plt.title('Clustering des descriptions de plats')
plt.show()

# 6. Analyse avec Transformers (Hugging Face) pour classification ou extraction
# Exemple : classification de sentiment
classifier = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment')

for desc in descriptions[:5]:
    result = classifier(desc)
    print(f"Description: {desc}\nSentiment: {result}\n")

ModuleNotFoundError: No module named 'numpy.strings'

In [ ]:
import requests
from bs4 import BeautifulSoup

import nltk
from nltk.corpus import stopwords

from gensim.models import Word2Vec

from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import seaborn as sns

import spacy
nlp = spacy.load('fr_core_news_sm')

# Téléchargements nécessaires
nltk.download('stopwords')

# 1. Récupérer des descriptions de plats depuis une page web
url = 'https://huggingface.co/datasets/worldcuisines/food-kb-v1.1'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Supposons que les descriptions sont dans des balises <p>
descriptions = [p.get_text() for p in soup.find_all('p')]

# 2. Prétraitement du texte
nlp = spacy.load('fr_core_news_sm')  # Modèle français, ou 'en_core_web_sm' pour anglais

def preprocess_text(text):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return tokens

processed_descriptions = [preprocess_text(desc) for desc in descriptions]

# 3. Extraction de caractéristiques avec Word2Vec (Gensim)
model_w2v = Word2Vec(sentences=processed_descriptions, vector_size=100, window=5, min_count=2, workers=4)

# Obtenir une représentation vectorielle moyenne pour chaque description
import numpy as np

def get_sentence_vector(tokens, model):
    valid_tokens = [token for token in tokens if token in model.wv]
    if not valid_tokens:
        return np.zeros(model.vector_size)
    return np.mean([model.wv[token] for token in valid_tokens], axis=0)

vectors = np.array([get_sentence_vector(tokens, model_w2v) for tokens in processed_descriptions])

# 4. Clustering avec scikit-learn
#kmeans = KMeans(n_clusters=3, random_state=42)
#clusters = kmeans.fit_predict(vectors)
# Si tu veux continuer avec un seul vecteur, utilise n_clusters=1
kmeans = KMeans(n_clusters=1, random_state=42)
clusters = kmeans.fit_predict(vectors)

# 5. Visualisation avec PCA
pca = PCA(n_components=1)
reduced_vectors = pca.fit_transform(vectors)

plt.figure(figsize=(8,6))
##sns.scatterplot(x=reduced_vectors[:,0], y=reduced_vectors[:,1], hue=clusters, palette='Set2')
# Ensuite, fais le scatterplot seulement si tu as bien 2 dimensions
if reduced_vectors.shape[1] == 2:
    import seaborn as sns
    import matplotlib.pyplot as plt

    plt.figure(figsize=(8,6))
    sns.scatterplot(x=reduced_vectors[:,0], y=reduced_vectors[:,1], hue=clusters, palette='Set2')
    plt.title('Clustering des descriptions de plats')
    plt.show()
else:
    print("Impossible de faire un scatter plot : le vecteur réduit n'a qu'une dimension.")

plt.title('Clustering des descriptions de plats')
plt.show()

# 6. Analyse avec Transformers (Hugging Face) pour classification ou extraction
# Exemple : classification de sentiment
classifier = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment')

for desc in descriptions[:5]:
    result = classifier(desc)
    print(f"Description: {desc}\nSentiment: {result}\n")

In [ ]:
import requests
from bs4 import BeautifulSoup

import nltk
from nltk.corpus import stopwords

from gensim.models import Word2Vec

from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import seaborn as sns

import spacy
nlp = spacy.load('fr_core_news_sm')

# Téléchargements nécessaires
nltk.download('stopwords')

# 1. Récupérer des descriptions de plats depuis une page web (exemple)
url = 'https://huggingface.co/datasets/worldcuisines/food-kb'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Supposons que les descriptions sont dans des balises <p>
descriptions = [p.get_text() for p in soup.find_all('p')]

# 2. Prétraitement du texte
nlp = spacy.load('fr_core_news_sm')  # Modèle français, ou 'en_core_web_sm' pour anglais

def preprocess_text(text):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return tokens

processed_descriptions = [preprocess_text(desc) for desc in descriptions]

# 3. Extraction de caractéristiques avec Word2Vec (Gensim)
model_w2v = Word2Vec(vector_size=100, window=5, min_count=2, workers=4)
model_w2v.build_vocab(processed_descriptions)
#model_w2v.train(processed_descriptions, total_examples=len(processed_descriptions), epochs=10)

#model_w2v = Word2Vec(sentences=processed_descriptions, vector_size=100, window=5, min_count=2, workers=4)

# Obtenir une représentation vectorielle moyenne pour chaque description
import numpy as np

def get_sentence_vector(tokens, model):
    valid_tokens = [token for token in tokens if token in model.wv]
    if not valid_tokens:
        return np.zeros(model.vector_size)
    return np.mean([model.wv[token] for token in valid_tokens], axis=0)

vectors = np.array([get_sentence_vector(tokens, model_w2v) for tokens in processed_descriptions])

# 4. Clustering avec scikit-learn
#kmeans = KMeans(n_clusters=3, random_state=42)
#clusters = kmeans.fit_predict(vectors)
import numpy as np

vectors = np.array(vectors)
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(vectors)

# 5. Visualisation avec PCA
pca = PCA(n_components=2)
reduced_vectors = pca.fit_transform(vectors)

plt.figure(figsize=(8,6))
sns.scatterplot(x=reduced_vectors[:,0], y=reduced_vectors[:,1], hue=clusters, palette='Set2')
plt.title('Clustering des descriptions de plats')
plt.show()

# 6. Analyse avec Transformers (Hugging Face) pour classification ou extraction
# Exemple : classification de sentiment
classifier = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment')

for desc in descriptions[:5]:
    result = classifier(desc)
    print(f"Description: {desc}\nSentiment: {result}\n")